In [3]:
import numpy as np
import random
import os
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

#from keras import backend as K


In [2]:
# Dataset path
path="/Users/fgimbert/Documents/Dataset/Manual/Preprocessed/"


In [3]:
# load dataset images in numpy and save if for first time

def load_into_np(folderPictures=path):
    index = 0
    dataset = []
    unknown = []
    labels = []
    folders=next(os.walk(folderPictures))[1]
    print(folders)
    img_x = 50
    img_y = 75

    for folder in folders[:1]:
        for img_file in os.listdir(folderPictures+folder):
            #print(img_file)
            name, label = img_file.strip('.png').split("_")
            #print(name, label)
            if label == 'UNKNOWN':
                picture = mpimg.imread(path +folder+'/'+img_file).reshape(img_x * img_y)
                unknown.append(picture)
                #print('Unknown !!')
            else:
                picture = mpimg.imread(path +folder+'/'+img_file).reshape(img_x * img_y)
            #print(picture.shape)
            #picture = path + str(repertory) + "/" + str(file) + "_" + str(label) + ".png"
                dataset.append(picture)
                labels.append(label)
    
    dataset = np.array(dataset)
    #labels = np.array(labels).reshape(-1,1)
    #print(labels.shape)
    
    import pickle

    with open('dataset.pkl', 'wb') as fp:
        pickle.dump(dataset, fp)
        
    with open('labels.pkl', 'wb') as fp:
        pickle.dump(labels, fp)
        

    
    #np.savetxt('dataset.csv', dataset, delimiter=",")
    #np.savetxt('labels.csv', labels, delimiter=",")

    
            

In [20]:
load_into_np(path)

NameError: name 'load_into_np' is not defined

In [19]:
print(len(np.load('labels.pkl')))
print(np.load('dataset.pkl').shape)

#Only keep the hieroglyph classes with 2 examples. 

count_labels = {}

full_labels = np.load('labels.pkl')
full_dataset = np.load('dataset.pkl')

for label in full_labels:
    if label in count_labels:
        count_labels[label] += 1
    else:
        count_labels[label] = 1

labels = []
dataset = full_dataset[0,:].reshape(1,-1)
#print(full_dataset[0,:].reshape(1,-1).shape)
#print(dataset.shape)

for i, label in enumerate(full_labels):
    if count_labels[label] > 1:
        labels.append(label)
        dataset = np.concatenate((dataset, full_dataset[i,:].reshape(1,-1)), axis=0)
 
dataset = np.delete(dataset, (0), axis=0)  

print(dataset.shape)
print(len(labels))

361
(361, 3750)


(333, 3750)
333


In [ ]:
# Create train and test dataset 
# Hieroglyph in test dataset if only count_labels[hiero] > 2 



In [ ]:
def euclidean_distance(vects):
    x, y = vects
    sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))


def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)


def contrastive_loss(y_true, y_pred):
    '''Contrastive loss from Hadsell-et-al.'06
    http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    '''
    margin = 1
    square_pred = K.square(y_pred)
    margin_square = K.square(K.maximum(margin - y_pred, 0))
    return K.mean(y_true * square_pred + (1 - y_true) * margin_square)


def create_pairs(x, digit_indices):
    '''Positive and negative pair creation.
    Alternates between positive and negative pairs.
    '''
    pairs = []
    labels = []
    n = min([len(digit_indices[d]) for d in range(num_classes)]) - 1
    for d in range(num_classes):
        for i in range(n):
            z1, z2 = digit_indices[d][i], digit_indices[d][i + 1]
            pairs += [[x[z1], x[z2]]]
            inc = random.randrange(1, num_classes)
            dn = (d + inc) % num_classes
            z1, z2 = digit_indices[d][i], digit_indices[dn][i]
            pairs += [[x[z1], x[z2]]]
            labels += [1, 0]
    return np.array(pairs), np.array(labels)